# BlueSprig-CV 蓝枝简历 — 简单demo演示
## 1. 概述
    本demo演示了本组使用蓝心大模型，通过构建prompt和message，实现了部分最终产品功能的python代码演示

主要功能
1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式
2. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化
3. 根据规范化简历json，基于场景构建一场模拟面试，用语音播放题目，允许用户使用语音或文字回答问题，并在最终给出面试评分
    1. 面试场景可以分为：技术面试、产品面试、HR面试、技术+产品面试、技术+HR面试等等，后续可能根据不同行业标准进行调整，可以通过RAG减弱大模型幻觉现象，保证面试题更贴近真实面试，（本功能点因为时间问题当前demo并未实现）


## 2. 主要功能演示
### 1. 解析word，pdf或者图片格式的简历，给出构建后的规范化简历json格式

In [ ]:
import json

# 工具方法
from util.file_loader import load_file
from util.llm import format_resume


# 定义一个函数，用于读取txt文件
def read_txt(file_path):
    # 打开文件，以只读模式，编码格式为utf-8
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取文件内容，并去除首尾空格
        return file.read().strip()

In [ ]:
# 读取简历文件
resume_path = 'assets/pdf/1.pdf'
content = load_file(resume_path)
content

In [ ]:
# 获取标准解析简历json格式
resume_format = read_txt('assets/txt/resume_format.txt')
response_content = format_resume(resume_format, content)
response_content

### 3. 根据规范化简历json，生成当前简历各个模块的评分，优化后的版本以及优化建议，提供多轮对话，可以按照用户的想法继续优化

#### 3.1. 简历优化版本

In [ ]:
# 简历json数据
resume_json = json.loads(response_content)
resume_json

In [ ]:
from util.llm import better_resume_by_module


work_experience_content = resume_json["work_experience"]
education_experience_content = resume_json["education_experience"]
project_experience_content = resume_json["project_experience"]

work_experience_format = read_txt("assets/txt/greater_resume_module/work_experience.txt")
education_experience_format = read_txt("assets/txt/greater_resume_module/education_experience.txt")
project_experience_format = read_txt("assets/txt/greater_resume_module/project_experience.txt")

work = better_resume_by_module(work_experience_format, work_experience_content, "工作经历")
education = better_resume_by_module(education_experience_format, education_experience_content, "教育经历")
project = better_resume_by_module(project_experience_format, project_experience_content, "项目经历")

project = json.loads(project)
work = json.loads(work)
education = json.loads(education)

resume_json["project_experience"] = project
resume_json["work_experience"] = work
resume_json["education_experience"] = education

resume_json

#### 3.2. 简历优化对话

In [ ]:
# 获取原始简历
resume_json_old = json.loads(response_content)
resume_json_old

In [ ]:
# 获取优化后的简历
resume_json_new = resume_json
resume_json_new

In [ ]:
# 介于jupyter的限制，这里采用的预设好的用户问题对话，我们也制作的终端对话的脚本，请参照show_main.py

### 4. 根据规范化简历json，基于场景构建一场模拟面试，用语音播放题目，允许用户使用语音或文字回答问题，并在最终给出面试评分
#### 4.1. 模拟面试

In [ ]:
# 获取简历json
resume_json

In [ ]:
from util.llm import generate_interview_questions

# 生成面试题列表
interview_questions = generate_interview_questions(resume_json)
questions_list = json.loads(interview_questions)["questions"]
questions_list

In [ ]:
from util.llm import generate_questions_audio

# 生成面试题语音
questions_audio_list = generate_questions_audio(questions_list)
questions_audio_list

#### 4.2. 面试评分

In [ ]:
# 处理用户答案，这里还是采用预设的问题和结果进行演示
user_answers = [
    {
        "answer":"",
        "type": "text"
    },
    {
        "answer":"",
        "type": "audio"
    }
]

answers_list = []
answers_result = []
for i in range(len(questions_list)):
    answers_result.append({
        "question": questions_list[i],
        "answer": answers_list[i]
    })

answers_result

In [ ]:
# 面试结果
from util.llm import interview_result
interview_result_format = read_txt("assets/txt/interview_result_format.txt")
interview_result = interview_result(interview_result_format, resume_json, answers_result)
interview_result